In [1]:
import asyncio 

from src.llm.gigachat_class import GigaChat
from src.llm.prompts import TAB_FACT_PROMPT, WIKITQ_PROMPT

from src.utils.dataset_loader import *
from src.utils.table_formatters import *

### Model

In [2]:
giga = GigaChat()

/media/ntfs_disk/Pycharn/TabularData4LLM/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### Data

In [3]:
async def get_formats(data_sample, mode):
    headers, rows = data_sample['table']['header'], data_sample['table']['rows']
    df_ = array_to_df(headers, rows)
    html = convert_df_to_html(df_)
    rst = await convert_html_to_list_table(html)
    if mode == 'html':
        return html
    else:
        return rst

#### Tabfact

In [4]:
tabfact_data = load_tab_fact()['train']
fact_dicts = []
for i in tabfact_data:
    d = {}
    d.update(i)
    fact_dicts.append(d)

human_message_template = """
Таблица: {table}
Утверждение: {statement}
"""
dataset = 'tabfact'

CHOSEN_PROMPT = TAB_FACT_PROMPT

#### WikiTQ

In [5]:
wikitq_data = load_wikitq()['train']
fact_dicts = []
for i in wikitq_data:
    d = {}
    d.update(i)
    d['statement'] = d['question']
    d['label'] = d['answers'][0]
    fact_dicts.append(d)

human_message_template = """
Таблица: {table}
Вопрос: {statement}
"""
dataset = 'wikitq'

CHOSEN_PROMPT = WIKITQ_PROMPT

### Eval

In [7]:
import os
import pickle 
from tqdm import tqdm

from src.config import giga_config

BATCH_SIZE=10
mode = 'html'
os.makedirs(f'results/{dataset}/{giga_config.GIGACHAT_MODEL}/{mode}', exist_ok=True)
for start in tqdm(range(0, len(fact_dicts[:2000]), BATCH_SIZE)):
    end = min(start + BATCH_SIZE, len(fact_dicts))
    
    statements = [x['statement'] for x in fact_dicts[start:end]]
    labels = [x['label'] for x in fact_dicts[start:end]]
    tables = [await get_formats(x, mode) for x in fact_dicts[start:end]]
    
    if statements:
        tasks = [giga.async_ask_giga(human_message_template.format(table=table, statement=statement), prompt=CHOSEN_PROMPT) for table, statement in zip(tables, statements)]
        results = await asyncio.gather(*tasks)
        with open(f'results/{dataset}/{giga_config.GIGACHAT_MODEL}/{mode}/{start}_{end}.pkl', 'wb') as f:
            pickle.dump(results, f)

 59%|███████████████████████████████████████████████████████████████████████████████████▏                                                         | 59/100 [11:27<06:49, 10.00s/it]

An error occurred in Giga Module, f: async_ask_giga, e: 'choices'


/media/ntfs_disk/Pycharn/TabularData4LLM/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


An error occurred in Giga Module, f: async_ask_giga, e: 'choices'


/media/ntfs_disk/Pycharn/TabularData4LLM/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

/media/ntfs_disk/Pycharn/TabularData4LLM/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


An error occurred in Giga Module, f: async_ask_giga, e: 'choices'


/media/ntfs_disk/Pycharn/TabularData4LLM/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


An error occurred in Giga Module, f: async_ask_giga, e: 'choices'


/media/ntfs_disk/Pycharn/TabularData4LLM/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [18:42<00:00, 11.22s/it]


### Metrics

In [8]:
import os
import re
import pickle 

In [11]:
i = 0

tp_score = 0
cnt_unparsed_answer = 0
mode = 'html'

base_file_path = f'results/{dataset}/{giga_config.GIGACHAT_MODEL}/{mode}'
for start in tqdm(range(0, len(fact_dicts[:2000]), BATCH_SIZE)):
    end = min(start + BATCH_SIZE, len(fact_dicts))
    labels = [x['label'] for x in fact_dicts[start:end]]
    with open(os.path.join(base_file_path, f"{start}_{end}.pkl"), 'rb') as f:
        model_answers = pickle.load(f)
    
    res_on_batch = []
    for case_answer in model_answers:
        if not case_answer:
            cnt_unparsed_answer += 1
            res_on_batch.append(2)
            continue
        answer = re.search(r'([Ии]тоговый ответ: .*)|([Ff]inal [Aa]nswer: .*)' ,case_answer)
        if answer:
            answer = answer.group().split(":")[-1].strip()
            res_on_batch.append(answer)
        
        else:
            cnt_unparsed_answer += 1
            res_on_batch.append(2)
    
    for predicted, gt in zip(res_on_batch, labels):
        tp_score += predicted == str(gt)
print(tp_score, tp_score/(2000-cnt_unparsed_answer), cnt_unparsed_answer)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 630.40it/s]

709 0.38365800865800864 152
